In [1]:
import os
from pathlib import Path
import sys
sys.path.insert(0, str(Path(os.getcwd()).parent))

In [2]:
import logging
from financerag.retrieval import BM25Retriever, BM25Processor
import financerag.tasks as tasks_module

import importlib
import inspect
import os
import json
import pandas as pd
from datasets import load_dataset

from nltk.tokenize import word_tokenize, TweetTokenizer
from rank_bm25 import BM25Okapi
import nltk

tweet_tokenizer = TweetTokenizer()
logging.basicConfig(level=logging.INFO)

/opt/homebrew/Caskroom/miniforge/base/envs/financerag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
retrieval = ['FinDER', 'FinQABench', 'FinanceBench', 'TATQA', 'FinQA', 'ConvFinQA', 'MultiHiertt']
tabular_retrieval = ['TATQA', 'FinQA', 'ConvFinQA', 'MultiHiertt']

In [4]:
from typing import Any, Callable, Dict, List, Literal, Optional

def tokenize_list_tweet(input_list: List[str]) -> List[List[str]]:
    """
    Tokenizes a list of strings using the `nltk.word_tokenize` function.

    Args:
        input_list (`List[str]`):
            A list of input strings to be tokenized.

    Returns:
        `List[List[str]]`:
            A list where each element is a list of tokens corresponding to an input string.
    """
    return list(map(tweet_tokenizer.tokenize, input_list))

In [5]:
output_dir = './BM25'
os.makedirs(output_dir, exist_ok=True)

def process_task(task_class, corpus_documents, output_dir, finder_task):
    # Tokenize corpus
    processor = BM25Processor(tokenizer=tweet_tokenizer.tokenize)
    # Tokenize and prepare the corpus
    tokenized_corpus = processor.build_corpus(corpus_documents)
    
    print(len(tokenized_corpus))
    # tokenized_corpus = [tweet_tokenizer.tokenize(doc) for doc in corpus_documents]
    
    # Initialize BM25 and retrieval model
    bm25_model = BM25Okapi(tokenized_corpus)
    
    retrieval_model = BM25Retriever(bm25_model, tokenize_list_tweet)
    
    # Retrieve documents
    retrieval_result = finder_task.retrieve(
        retriever=retrieval_model,
        # top_k=500
    )
    
    # Save retrieval result
    file_name = f"{output_dir}/{task_class}.json"
    with open(file_name, "w") as json_file:
        json.dump(retrieval_result, json_file, indent=4)
    
    # Save evaluation result
    df = pd.read_csv(f'../data/{task_class.split("_")[0]}_qrels.tsv', sep='\t')
    qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
    
    eval_result = finder_task.evaluate(qrels_dict, retrieval_result, [1, 5, 10])
    combined_result = {**eval_result[0], **eval_result[1], **eval_result[2], **eval_result[3]}
    df_eval = pd.DataFrame([combined_result])
    df_eval.to_csv(f'{output_dir}/{task_class.split("_")[0]}_eval.csv', index=False)


In [12]:
processor = BM25Processor(tokenizer=tweet_tokenizer.tokenize)
# Tokenize and prepare the corpus
tokenized_corpus = processor.build_corpus(corpus)

In [17]:
corpus_list = [{"title": value["title"], "text": value["text"]} for value in finder_task.corpus.values()]


In [19]:
token = processor.build_corpus(corpus_list)
len(token)

13863

In [6]:
task_class = 'FinDER'
task_class_obj = getattr(tasks_module, task_class)
finder_task = task_class_obj()

# corpus = pd.read_csv(f"./data/{task_class}_corpus_convert.csv")
corpus = load_dataset("Linq-AI-Research/FinanceRAG", task_class, split="corpus")
# process_task(task_class, corpus, output_dir, finder_task)

INFO:financerag.common.loader:Loading Corpus...
INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsi

In [11]:
finder_task.corpus

{'ADBE20230004': {'title': 'ADBE OVERVIEW',
  'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsible. Our products and services help unleash creativity, accelerate document productivity and power businesses in a digital world.'},
 'ADBE20230006': {

In [6]:

for task_class in retrieval:
    print(f"Running for {task_class}")
    if task_class in tabular_retrieval:
        pass
    task_class_obj = getattr(tasks_module, task_class)
    finder_task = task_class_obj()

    # corpus = pd.read_csv(f"./data/{task_class}_corpus_convert.csv")
    corpus = load_dataset("Linq-AI-Research/FinanceRAG", task_class, split="corpus")
    process_task(task_class, corpus, output_dir, finder_task)

INFO:financerag.common.loader:Loading Corpus...


Running for FinDER


INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsible. Our products and services help unleash crea

13867


/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/382860926.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.0000
INFO:financerag.tasks.BaseTask:NDCG@5: 0.0048
INFO:financerag.tasks.BaseTask:NDCG@10: 0.0116
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP@1: 0.0000
INFO:financerag.tasks.BaseTask:MAP@5: 0.0026
INFO:financ

...

/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/382860926.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.2381
INFO:financerag.tasks.BaseTask:NDCG@5: 0.3824
INFO:financerag.tasks.BaseTask:NDCG@10: 0.4224
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP@1: 0.2381
INFO:financerag.tasks.BaseTask:MAP@5: 0.3452
INFO:financ

Running for MultiHiertt


INFO:financerag.common.loader:Loaded 10475 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'd8e4ea4ac', 'title': '', 'text': '|  | Years Ended December 31, |\n|  | 2006 | 2005 |\n|  | (In millions) |\n| Investment return | $192 | $-26 |\n| Expense | 45 | 11 |\n| In-force/Persistency | -7 | -33 |\n| Policyholder dividends and other | -39 | -11 |\n| Total | $191 | $-59 |\nAs of December 31, 2006 and 2005, DAC and VOBA for the Individual segment were $14.0 billion and $13.5 billion, respectively, and for the total Company were $20.8 billion and $19.7 billion, respectively.\nGoodwill Goodwill is the excess of cost over the fair value of net assets acquired.\nThe Company tests goodwill for impairment at least annually or more frequently if events or circumstances, such as adverse changes in the business climate, indicate that there may be justification for conducting an interim test.\nImpairment testing is performed using the fair value approach, which requires the use of es

10475


INFO:financerag.retrieval.bm25:Tokenizing queries with lower cases
/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/382860926.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.0822
INFO:financerag.tasks.BaseTask:NDCG@5: 0.0576
INFO:financerag.tasks.BaseTask:NDCG@10: 0.0643
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:MAP

In [19]:
output_dir = './BM25'

master_result = pd.DataFrame()
for task_class in retrieval:
    df_eval = pd.read_csv(f'{output_dir}/{task_class.split("_")[0]}_eval.csv')
    df_eval['task'] = task_class
    master_result = pd.concat([master_result, df_eval])
master_result.to_csv(f'{output_dir}/master_eval.csv')


In [20]:
master_result

,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,Recall@1,Recall@5,Recall@10,P@1,P@5,P@10,task
0,0.00000,0.00479,0.01156,0.00000,0.00260,0.00499,0.00000,0.00781,0.02734,0.00000,0.00313,0.00469,FinDER
0,0.56667,0.71819,0.74058,0.56667,0.67000,0.67972,0.56667,0.86667,0.93333,0.56667,0.17333,0.09333,FinQABench
0,0.08889,0.14304,0.16246,0.06667,0.11741,0.12552,0.06667,0.20000,0.25556,0.08889,0.04444,0.02889,FinanceBench
0,0.16466,0.23783,0.26463,0.16466,0.21493,0.22609,0.16466,0.30723,0.38956,0.16466,0.06145,0.03896,TATQA
0,0.28488,0.42283,0.45848,0.28488,0.38261,0.39783,0.28488,0.54360,0.65116,0.28488,0.10872,0.06512,FinQA
0,0.23810,0.38241,0.42243,0.23810,0.34524,0.36115,0.23810,0.49206,0.61905,0.23810,0.09841,0.06190,ConvFinQA
0,0.08219,0.05764,0.06433,0.02203,0.03245,0.03506,0.02203,0.04995,0.06830,0.08219,0.03904,0.02671,MultiHiertt


In [14]:
output_dir = './BM25_ms-marco-MiniLM-L-12-v2'
os.makedirs(output_dir, exist_ok=True)
from financerag.rerank import CrossEncoderReranker
from sentence_transformers import CrossEncoder

# reranker = CrossEncoderReranker(
#     model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
# )
def process_task_reranker(task_class, corpus_documents, output_dir, finder_task):
    # Tokenize corpus
    reranker = CrossEncoderReranker(
    model=CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
    )
    processor = BM25Processor(tokenizer=tweet_tokenizer.tokenize)
    # Tokenize and prepare the corpus
    tokenized_corpus = processor.build_corpus(corpus_documents)
    
    print(len(tokenized_corpus))
    # tokenized_corpus = [tweet_tokenizer.tokenize(doc) for doc in corpus_documents]
    
    # Initialize BM25 and retrieval model
    bm25_model = BM25Okapi(tokenized_corpus)
    
    retrieval_model = BM25Retriever(bm25_model, tokenize_list_tweet)
    
    # Retrieve documents
    retrieval_result = finder_task.retrieve(
        retriever=retrieval_model,
        # top_k=500
    )
    reranking_result = finder_task.rerank(
                        reranker=reranker,
                        results=retrieval_result,
                        top_k=100,  # Rerank the top 100 documents
                        batch_size=32
                    )
    # Save retrieval result
    file_name = f"{output_dir}/{task_class}.json"
    with open(file_name, "w") as json_file:
        json.dump(reranking_result, json_file, indent=4)
    
    # Save evaluation result
    df = pd.read_csv(f'../data/{task_class.split("_")[0]}_qrels.tsv', sep='\t')
    qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
    
    eval_result = finder_task.evaluate(qrels_dict, reranking_result, [1, 5, 10])
    combined_result = {**eval_result[0], **eval_result[1], **eval_result[2], **eval_result[3]}
    df_eval = pd.DataFrame([combined_result])
    df_eval.to_csv(f'{output_dir}/{task_class.split("_")[0]}_eval.csv', index=False)


In [15]:
for task_class in retrieval:
    print(f"Running for {task_class}")
    if task_class in tabular_retrieval:
        pass
    task_class_obj = getattr(tasks_module, task_class)
    finder_task = task_class_obj()

    # corpus = pd.read_csv(f"./data/{task_class}_corpus_convert.csv")
    corpus = load_dataset("Linq-AI-Research/FinanceRAG", task_class, split="corpus")
    process_task_reranker(task_class, corpus, output_dir, finder_task)

INFO:financerag.common.loader:Loading Corpus...


Running for FinDER


INFO:financerag.common.loader:Loaded 13867 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'ADBE20230004', 'title': 'ADBE OVERVIEW', 'text': 'Adobe is a global technology company with a mission to change the world through personalized digital experiences. For over four decades, Adobe’s innovations have transformed how individuals, teams, businesses, enterprises, institutions, and governments engage and interact across all types of media. Our products, services and solutions are used around the world to imagine, create, manage, deliver, measure, optimize and engage with content across surfaces and fuel digital experiences. We have a diverse user base that includes consumers, communicators, creative professionals, developers, students, small and medium businesses and enterprises. We are also empowering creators by putting the power of artificial intelligence (“AI”) in their hands, and doing so in ways we believe are responsible. Our products and services help unleash crea

13867


INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches: 100%|██████████| 675/675 [06:58<00:00,  1.61it/s]
/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/3529174673.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.1094
INFO:financerag.tasks.BaseTask:NDCG@5: 0.1417
INFO:financerag.tasks.BaseTask:NDCG@10: 0.1446
INFO:f

...

INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches: 100%|██████████| 1316/1316 [12:48<00:00,  1.71it/s]
/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/3529174673.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.2143
INFO:financerag.tasks.BaseTask:NDCG@5: 0.3376
INFO:financerag.tasks.BaseTask:NDCG@10: 0.3818
INFO

Running for MultiHiertt


INFO:financerag.common.loader:Loaded 10475 Documents.
INFO:financerag.common.loader:Corpus Example: {'id': 'd8e4ea4ac', 'title': '', 'text': '|  | Years Ended December 31, |\n|  | 2006 | 2005 |\n|  | (In millions) |\n| Investment return | $192 | $-26 |\n| Expense | 45 | 11 |\n| In-force/Persistency | -7 | -33 |\n| Policyholder dividends and other | -39 | -11 |\n| Total | $191 | $-59 |\nAs of December 31, 2006 and 2005, DAC and VOBA for the Individual segment were $14.0 billion and $13.5 billion, respectively, and for the total Company were $20.8 billion and $19.7 billion, respectively.\nGoodwill Goodwill is the excess of cost over the fair value of net assets acquired.\nThe Company tests goodwill for impairment at least annually or more frequently if events or circumstances, such as adverse changes in the business climate, indicate that there may be justification for conducting an interim test.\nImpairment testing is performed using the fair value approach, which requires the use of es

10475


INFO:financerag.retrieval.bm25:Tokenizing queries with lower cases
INFO:financerag.rerank.cross_encoder:Starting To Rerank Top-100....
Batches: 100%|██████████| 3044/3044 [32:20<00:00,  1.57it/s]
/var/folders/7s/bvgzt9qx1mz2fl1nm48s5z3c0000gn/T/ipykernel_24904/3529174673.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qrels_dict = df.groupby('query_id').apply(lambda x: dict(zip(x['corpus_id'], x['score']))).to_dict()
INFO:financerag.tasks.BaseTask:For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
INFO:financerag.tasks.BaseTask:

INFO:financerag.tasks.BaseTask:NDCG@1: 0.1507
INFO:financerag.tasks.BaseTask

In [16]:
master_result = pd.DataFrame()
for task_class in retrieval:
    df_eval = pd.read_csv(f'{output_dir}/{task_class.split("_")[0]}_eval.csv')
    df_eval['task'] = task_class
    master_result = pd.concat([master_result, df_eval])

In [18]:
master_result.to_csv(f'{output_dir}/master_eval.csv')
